## This one works on a csv that ChatGPT gave me so I can't redo it. Anyway it wasn't able to save all pdfs.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse
from pathlib import Path

# Get the path to the current notebook
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()

# Navigate to the file: one level up, then into datasets
csv_path = notebook_dir.parent / "datasets" / "open_access_paper_links_GPTMADE.csv"

df = pd.read_csv(csv_path)
os.makedirs("papers", exist_ok=True)

for i, row in df.iterrows():
    doi = row["DOI"].replace("/", "_")
    url = row["Open Access URL"]

    try:
        # Try downloading as PDF directly
        if url.endswith(".pdf"):
            r = requests.get(url)
            if r.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r.content)
                print(f"✅ Downloaded: {doi}")
                continue

        # Otherwise, try to find a PDF link on the landing page
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")

        # Search for PDF links
        pdf_links = [a['href'] for a in soup.find_all('a', href=True) if "pdf" in a['href'].lower()]
        if pdf_links:
            pdf_url = pdf_links[0]
            if not pdf_url.startswith("http"):
                base = "{uri.scheme}://{uri.netloc}".format(uri=urlparse(url))
                pdf_url = base + pdf_url
            r_pdf = requests.get(pdf_url)
            if r_pdf.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r_pdf.content)
                print(f"✅ Scraped PDF from page: {doi}")
            else:
                print(f"⚠️ PDF link failed: {doi}")
        else:
            print(f"⚠️ No PDF found on page: {doi}")

    except Exception as e:
        print(f"❌ Error for {doi}: {e}")


⚠️ PDF link failed: 10.1186_s13731-025-00463-2
⚠️ PDF link failed: 10.1186_s41239-025-00511-7
⚠️ No PDF found on page: 10.1016_j.caeai.2025.100382
⚠️ No PDF found on page: 10.1016_j.respol.2025.105190
⚠️ No PDF found on page: 10.1016_j.rcim.2024.102937
⚠️ No PDF found on page: 10.1016_j.compbiomed.2025.110153
⚠️ PDF link failed: 10.1140_epjds_s13688-025-00547-9
⚠️ No PDF found on page: 10.1016_j.eswa.2025.126575
⚠️ No PDF found on page: 10.1016_j.technovation.2025.103208
⚠️ No PDF found on page: 10.3390_buildings15071095
⚠️ No PDF found on page: 10.3390_s25071958
⚠️ No PDF found on page: 10.1002_jee.20628
⚠️ No PDF found on page: 10.3390_agriengineering7040102
⚠️ No PDF found on page: 10.1016_j.chbr.2025.100652
✅ Downloaded: 10.1007_s00146-024-01959-3
⚠️ No PDF found on page: 10.1016_j.spc.2024.12.012
✅ Downloaded: 10.1007_s10798-024-09898-2
⚠️ No PDF found on page: 10.3390_educsci15030393
⚠️ No PDF found on page: 10.1016_j.adapen.2025.100211
⚠️ PDF link failed: 10.1617_s11527-025-0259

In [ ]:
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup

# Create directory for saving PDFs
os.makedirs("papers", exist_ok=True)

# Test URL and filename
url = "https://kth.diva-portal.org/smash/get/diva2:809845/FULLTEXT01"
doi = "diva2_809845_FULLTEXT01"

def download_pdf(doi, url):
    try:
        if url.endswith(".pdf"):
            r = requests.get(url)
            if r.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r.content)
                print(f"✅ Downloaded: {doi}")
                return

        # Scrape for PDF links
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        pdf_links = [a['href'] for a in soup.find_all('a', href=True) if "pdf" in a['href'].lower()]
        if pdf_links:
            pdf_url = pdf_links[0]
            if not pdf_url.startswith("http"):
                base = "{uri.scheme}://{uri.netloc}".format(uri=urlparse(url))
                pdf_url = base + pdf_url
            r_pdf = requests.get(pdf_url)
            if r_pdf.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r_pdf.content)
                print(f"✅ Scraped PDF: {doi}")
            else:
                print(f"⚠️ PDF download failed: {doi}")
        else:
            print(f"⚠️ No PDF link found: {doi}")

    except Exception as e:
        print(f"❌ Error: {doi} – {e}")

# Run the test
download_pdf(doi, url)


⚠️ No PDF link found: diva2_809845_FULLTEXT01


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

os.makedirs("papers_1", exist_ok=True)

headers = {"User-Agent": "Mozilla/5.0"}

for i, row in df.iterrows():
    doi = row["DOI"].replace("/", "_")
    url = row["Open Access URL"]

    try:
        # Direct PDF link
        if url.endswith(".pdf"):
            r = requests.get(url, headers=headers)
            with open(f"papers/{doi}.pdf", "wb") as f:
                f.write(r.content)
            print(f"✅ Downloaded: {doi}")
            continue

        # Check landing page
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        pdf_links = [a['href'] for a in soup.find_all('a', href=True) if "pdf" in a['href'].lower()]

        if pdf_links:
            pdf_url = urljoin(url, pdf_links[0])
            r_pdf = requests.get(pdf_url, headers=headers)
            if r_pdf.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r_pdf.content)
                print(f"✅ Scraped PDF from page: {doi}")
            else:
                print(f"⚠️ PDF link failed: {doi}")
        else:
            print(f"⚠️ No PDF found on page: {doi}")

    except Exception as e:
        print(f"❌ Error for {doi}: {e}")


FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\open_access_paper_links.csv'

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os
import pandas as pd

os.makedirs("papers", exist_ok=True)

options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

for i, row in df.iterrows():
    doi = row["DOI"].replace("/", "_")
    url = row["Open Access URL"]

    try:
        driver.get(url)
        time.sleep(3)
        links = driver.find_elements("xpath", "//a[contains(@href, '.pdf')]")
        if links:
            pdf_url = links[0].get_attribute("href")
            r = requests.get(pdf_url)
            if r.status_code == 200:
                with open(f"papers/{doi}.pdf", "wb") as f:
                    f.write(r.content)
                print(f"✅ Selenium download: {doi}")
            else:
                print(f"⚠️ PDF link not accessible: {doi}")
        else:
            print(f"⚠️ No PDF link found (Selenium): {doi}")
    except Exception as e:
        print(f"❌ Error {doi}: {e}")

driver.quit()


✅ Selenium download: 10.1186_s13731-025-00463-2
✅ Selenium download: 10.1186_s41239-025-00511-7
⚠️ No PDF link found (Selenium): 10.1016_j.caeai.2025.100382
⚠️ No PDF link found (Selenium): 10.1016_j.respol.2025.105190
⚠️ No PDF link found (Selenium): 10.1016_j.rcim.2024.102937
⚠️ No PDF link found (Selenium): 10.1016_j.compbiomed.2025.110153
✅ Selenium download: 10.1140_epjds_s13688-025-00547-9
⚠️ No PDF link found (Selenium): 10.1016_j.eswa.2025.126575
⚠️ No PDF link found (Selenium): 10.1016_j.technovation.2025.103208
⚠️ No PDF link found (Selenium): 10.3390_buildings15071095
⚠️ No PDF link found (Selenium): 10.3390_s25071958
⚠️ No PDF link found (Selenium): 10.1002_jee.20628
⚠️ No PDF link found (Selenium): 10.3390_agriengineering7040102
⚠️ No PDF link found (Selenium): 10.1016_j.chbr.2025.100652
⚠️ No PDF link found (Selenium): 10.1007_s00146-024-01959-3
⚠️ No PDF link found (Selenium): 10.1016_j.spc.2024.12.012
⚠️ No PDF link found (Selenium): 10.1007_s10798-024-09898-2
⚠️ No PDF

KeyboardInterrupt: 

In [ ]:
import pdfplumber
import os

os.makedirs("texts", exist_ok=True)

for file in os.listdir("papers"):
    if file.endswith(".pdf"):
        pdf_path = os.path.join("papers", file)
        txt_path = os.path.join("texts", file.replace(".pdf", ".txt"))

        try:
            with pdfplumber.open(pdf_path) as pdf:
                text = "\n".join([page.extract_text() or "" for page in pdf.pages])
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"📝 Text extracted: {file}")
        except Exception as e:
            print(f"❌ Text extraction failed: {file} — {e}")
